In [ ]:
import sys
sys.path.append("../")
from packages.plotter import *
from packages.physicalObjects.physicalValues import *
from packages.BEMPP.singleTraceManager import SingleTraceManager

Specify the parameters of the acoustic simulation:
 1. `radius`: the radius of the bubble, in meter
 1. `scatteringPoint`: the location to evaluate the acoustic field
 1. `spacingMultiplier`: the bubble separation, in bubble radii between bubble centers
 1. `frequencyRange`: the range of frequency values, in Hertz
 1. `numberOfPoints`: number of points in the frequency sweep
 1. `numberOfBubblesPerSide`: number of bubbles $n$ in the $n \times n$ array
 1. `hFactor`: the width of the mesh elements, as a proportion of the bubble radius
 
The square array of $n \times n$ bubbles is located in the plane $x=0$ and centered in the global origin. All bubbles have the same radius. The surface of each bubble is meshed with a triangular grid.

In [ ]:
# Parameters:
radius = 0.0001 # in meters
scatteringPoint = np.array([[3*radius], [0], [0]])
spacingMultiplier = 8 # distance between centers = spacingMultiplier * radius
L = radius*spacingMultiplier
frequencyRange = (130, 160)
numberOfPoints = 10
numberOfBubblesPerSide = 2
hFactor = 1/6

In [ ]:
def execute():
    manager = SingleTraceManager(formulationType=2, logs=True)
    # 0 = Dirichlet Formulation
    # 1 = Neumann Formulation
    # 2 = PMCHWT Formulation -> only one that is accelerated
    # 3 = Muller Formulation
    
    
    initialYCoordinate = 0 - L*(numberOfBubblesPerSide-1)/2 
    initialZCoordinate = 0 - L*(numberOfBubblesPerSide-1)/2
    for i in range(numberOfBubblesPerSide):
        for j in range(numberOfBubblesPerSide):
            yCoordinate = initialYCoordinate + i*L
            zCoordinate = initialZCoordinate + j*L
            manager.setNewBubble(center=(0,yCoordinate,zCoordinate), size=radius, h=hFactor*radius, intCoordinates=(0,i,j))
            
    manager.calculateFieldsFromMultipleFrequencies(scatteringPoint, wavenumberRange=frequencyRange, numberOfPoints=numberOfPoints, spacing_multiplier=spacingMultiplier)
    return manager

The following cell executes the fast BEM algorithm. By default, the accelerated PMCHWT formulation is used. The results are stored in the folder `pickles` and can be visualized with `plotter.ipynb`. The results of new simulations for the same configuration are added to this database.

In [ ]:
manager = execute()